### 假设性文档嵌入（HyDE）在文档检索中的应用

#### 概述

本代码实现了一个用于文档检索的假设性文档嵌入（HyDE）系统。HyDE是一种创新方法，将查询问题转换为包含答案的假设性文档，旨在弥合向量空间中查询和文档分布之间的差距。

#### 动机

传统的检索方法常常难以处理简短查询与更长、更详细的文档之间的语义差距。HyDE通过将查询扩展为完整的假设性文档来解决这个问题，通过使查询表示与向量空间中的文档表示更相似，可能提高检索的相关性。

#### 关键组件

1. PDF处理和文本分块
2. 使用FAISS和OpenAI嵌入创建向量存储
3. 生成假设性文档的语言模型
4. 实现HyDE技术的自定义HyDERetriever类

#### 方法细节

### 文档预处理和向量存储创建

1. 处理PDF并将其分割成块。
2. 使用OpenAI嵌入创建FAISS向量存储，以实现高效的相似性搜索。

### 假设性文档生成

1. 使用语言模型（GPT-4）生成回答给定查询的假设性文档。
2. 生成过程由提示模板引导，确保假设性文档详细且与向量存储中使用的块大小相匹配。

### 检索过程

`HyDERetriever`类实现了以下步骤：

1. 使用语言模型从查询生成假设性文档。
2. 将假设性文档用作向量存储中的搜索查询。
3. 检索与此假设性文档最相似的文档。

#### 核心特性

1. 查询扩展：将简短查询转换为详细的假设性文档。
2. 灵活配置：允许调整块大小、重叠和检索文档的数量。
3. 与OpenAI模型集成：使用GPT-4生成假设性文档，使用OpenAI嵌入进行向量表示。

#### 这种方法的好处

1. 提高相关性：通过将查询扩展为完整文档，HyDE可能捕捉到更微妙和相关的匹配。
2. 处理复杂查询：对于可能难以直接匹配的复杂或多方面的查询特别有用。
3. 适应性：假设性文档生成可以适应不同类型的查询和文档领域。
4. 潜在的更好上下文理解：扩展的查询可能更好地捕捉原始问题背后的上下文和意图。

#### 实施细节

1. 使用OpenAI的ChatGPT模型生成假设性文档。
2. 采用FAISS在向量空间中进行高效的相似性搜索。
3. 允许轻松可视化假设性文档和检索结果。

#### 结论

假设性文档嵌入（HyDE）代表了文档检索的一种创新方法，解决了查询和文档之间的语义差距。通过利用先进的语言模型将查询扩展为假设性文档，HyDE有潜力显著提高检索的相关性，特别是对于复杂或微妙的查询。这种技术在理解查询意图和上下文至关重要的领域可能特别有价值，如法律研究、学术文献回顾或高级信息检索系统。


<div style="text-align: center;">

<img src="../images/HyDe.svg" alt="HyDe" style="width:40%; height:auto;">
</div>

### Import libraries

In [14]:
import os
import sys
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from rag.helper_functions import *
from rag.evaluation.evalute_rag import *

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define document(s) path

In [15]:
path = "../data/Understanding_Climate_Change.pdf"

### 定义HyDe检索器类 - 创建向量存储、生成假设性文档和检索

In [16]:
class HyDERetriever:
    def __init__(self, files_path, chunk_size=500, chunk_overlap=100):
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)

        self.embeddings = OpenAIEmbeddings()
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.vectorstore = encode_pdf(files_path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
    
        
        self.hyde_prompt = PromptTemplate(
            input_variables=["query", "chunk_size"],
            template="""Given the question '{query}', generate a hypothetical document that directly answers this question. The document should be detailed and in-depth.
            the document size has be exactly {chunk_size} characters.""",
        )
        self.hyde_chain = self.hyde_prompt | self.llm

    def generate_hypothetical_document(self, query):
        input_variables = {"query": query, "chunk_size": self.chunk_size}
        return self.hyde_chain.invoke(input_variables).content

    def retrieve(self, query, k=3):
        hypothetical_doc = self.generate_hypothetical_document(query)
        similar_docs = self.vectorstore.similarity_search(hypothetical_doc, k=k)
        return similar_docs, hypothetical_doc


### 创建一个 HyDe 的检索器实例

In [17]:
retriever = HyDERetriever(path)

### 例子

In [18]:
test_query = "What is the main cause of climate change?"
results, hypothetical_doc = retriever.retrieve(test_query)

### 绘制假设性文档和检索到的文档

In [19]:
docs_content = [doc.page_content for doc in results]

print("hypothetical_doc:\n")
print(text_wrap(hypothetical_doc)+"\n")
show_context(docs_content)

hypothetical_doc:

**The Main Cause of Climate Change**  Climate change primarily results from human activities, particularly the burning
of fossil fuels such as coal, oil, and natural gas. This process releases significant amounts of carbon dioxide (CO2)
and other greenhouse gases into the atmosphere. Deforestation exacerbates the issue by reducing the number of trees that
can absorb CO2. Industrial processes, agriculture, and waste management also contribute to greenhouse gas emissions.
These gases trap heat, leading to global warming, altered weather patterns, and severe environmental impacts,
necessitating urgent action to mitigate these effects.

Context 1:
predict future trends. The evidence overwhelmingly shows that recent changes are primarily 
driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: Causes of Climate Change  
Greenhouse Gases  
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Gre